# KoGPT-2를 활용한 한국어 언어 생성 모델 및 fine-tuning

- 출처: <텐서플로2와 머신러닝으로 시작하는 자연어처리> 7장 실습코드
- github: https://github.com/NLP-kr/tensorflow-ml-nlp-tf2-colab


## 환경 준비

In [ ]:
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/requirements.txt -O requirements.txt
!pip install -r requirements.txt
!pip install tensorflow==2.2.0

## 데이터 다운로드

In [ ]:
!mkdir -p data_in/KOR
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/finetune_data.txt \
              -O data_in/KOR/finetune_data.txt 

## 사전 학습 모델 불러오기

In [23]:
import os

import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

GPT2Model

- **_ init _** : TFGPT2LMHeadModel을 생성해서 실행할 수 있게 구현, transformer 생성 방식에 따라 모델 소스가 저장된 디렉토리 경로 입력
- **call**: 출력 값은 tuple -> (last_hidden_states, past, hidden_status, attentions)
    * **last_hidden_states**: 모델의 마지막 레이어에서 출력한 값
    * **past**: 각 레이어에서 연산한 결과값 출력 -> 다음 토큰 예측시 연산 속도 빠르게 함. (입력한 시퀀스의 결괏값이 이미 있어서 이전 토큰에 대한 연산을 할 필요가 없기 때문)
    * **hidden_states**: 전체 레이어에 대한 은닉 상태 벡터를 모두 출력한 값
    * **attention**: 모든 레이어에 연산한 어텐션 맵 값
- 생성 모델에 활용하기 위해서는 vocabulary에 대한 logit 값만 활용하도록 첫 번째 값인 last_hidden_states를 출력 -> self.gpt2(inputs)[0]

In [24]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

## 사전 학습 모델 문장 생성

In [ ]:
# 학습된 파라미터 다운 받기
!wget https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip -O gpt_ckpt.zip
!unzip -o gpt_ckpt.zip

In [ ]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH) # BASE_MODEL_PATH 경로의 학습된 파라미터를 가지는 GPT-2 모델 선언

# 제가 여러 가지 데이터셋으로 실험해보느라..
# exo_model = GPT2Model(BASE_MODEL_PATH)
# sunny_model = GPT2Model(BASE_MODEL_PATH)
# novel_model = GPT2Model(BASE_MODEL_PATH)

In [7]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30

# tokenizer
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'
tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)

# 단어 사전
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

|스페셜 토큰|역할|
|:-|:-|
|< unk >|모르는 단어에 대한 토큰|
|< pad >|배치 데이터 길이 맞추는 용도|
|< s >|문장의 시작을 알림|
|< /s >|문장의 종결을 알림|

In [10]:
def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)

    return sent

In [11]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])

In [19]:
generate_sent('우리', gpt_model, greedy=True)

'우리 모두는 그 어느 때보다 더 많은 것을 얻어야 한다.'

In [20]:
generate_sent('우리', gpt_model, top_k=0, top_p=0.95)

'우리보다 훨씬 여유있고 현명한 나였지만 동시에 우리 세대에 큰 영향을 미쳤던 자 그들에 대한 예의에 관해 생각해보면 어떨까, 하는 생각을 해본 적이 있다.'

## Fine Tuning

In [130]:
DATA_IN_PATH = './data_in/KOR/'
TRAIN_DATA_FILE = 'finetune_data.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE).readlines()]

In [131]:
input_data = []
output_data = []

# 텍스트 데이터 토큰화 과정
for s in sents:
    tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

In [132]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [133]:
novel_model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

In [ ]:
history = novel_model.fit(input_data, output_data, 
                    batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                    validation_split=0.1)

In [135]:
DATA_OUT_PATH = './data_out'
model_name = "tf2_gpt2_finetuned_model_novel"

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

novel_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at ./data_out/tf2_gpt2_finetuned_model_novel.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [98]:
# 저장된 모델 불러오기
# 이 코드는 필요할 때 사용하세요!

model_name = "model folder"
model_path = os.path.join(DATA_OUT_PATH, model_name)
model_model = GPT2Model(model_path)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at ./data_out/tf2_gpt2_finetuned_model_exo.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [144]:
def gpt_fine_tuning_test(word, k=0, p=0):
  print('GPT-2')
  print('greedy True: ', generate_sent(word, gpt_model, greedy=True))
  print('greedy False: ', generate_sent(word, gpt_model, top_k=k, top_p=p))
  print()
  print('EXO lyrics fine-tuning')
  print('greedy True: ', generate_sent(word, exo_model, greedy=True))
  print('greedy False: ', generate_sent(word, exo_model, top_k=k, top_p=p))
  print()
  print('Movie Sunny fine-tuning')
  print('greedy True: ', generate_sent(word, sunny_model, greedy=True))
  print('greedy False: ', generate_sent(word, sunny_model, top_k=k, top_p=p))
  print()
  print('Novel fine-tuning')
  print('greedy True: ', generate_sent(word, novel_model, greedy=True))
  print('greedy False: ', generate_sent(word, novel_model, top_k=k, top_p=p))

In [145]:
gpt_fine_tuning_test('사랑', p=0.95)

GPT-2
greedy True:  사랑합니다
greedy False:  사랑합니다 어서 오세요

EXO lyrics fine-tuning
greedy True:  사랑해요 마마마마마마마마마마마마마마마마마마마마마마마마마마마마마마마마마마마마마마마 Turn back
greedy False:  사랑해요 마마! 내 꺼야

Movie Sunny fine-tuning
greedy True:  사랑 가득하고 즐거운
greedy False:  사랑한다면 나이가 들수록 자연스럽게 늙는 법이지. 나이가 들수록 자연스럽게 늙는 법이지. 어<unk>히. 인생은 짧고 혁명은                                                                        

Novel fine-tuning
greedy True:  사랑한다면
greedy False:  사랑해요 내 말 들을 형편이 안 되는 내 맘 알아요 속상해도 이해해 줄 거야


In [155]:
gpt_fine_tuning_test('인간', p=0.95)

GPT-2
greedy True:  인간관계에서 가장 중요한 것은 인간관계에서 가장 중요한 것이 인간관계라는 것을 잊지 말아야 한다.
greedy False:  인간으로서의 기본 예의가 전혀 없는 것이다.

EXO lyrics fine-tuning
greedy True:  인간에게 있어서 가장 아름다운 이별은 무엇일까?
greedy False:  인간 사냥꾼은 없다"고 말한 뒤, 무언가 잃어버린 듯한 기분이 들었다.

Movie Sunny fine-tuning
greedy True:  인간 임 나미. 너는 임 나미? (장미에게) 나 조퇴 좀 하게....말투가 전라도 말투다.
greedy False:  인간 임 나미. 너 클때는 응? 다 멋있다. 이런 모습 처음이네. 응? 진희는? 황홀경

Novel fine-tuning
greedy True:  인간에게 있어서 가장 큰 고비는 무엇인가가 아닐까.
greedy False:  인간으로서 열등감 같은 것을 느끼기는 커녕, 아침 일찍 일어나기 위해 슬근슬근 논다느니보다 오히려 근심과 걱정이 앞선다.


In [148]:
하지만 gpt_fine_tuning_test('그녀', p=0.95)

GPT-2
greedy True:  그녀와 함께 있으면 행복할 것 같다.
greedy False:  그녀 손목이!

EXO lyrics fine-tuning
greedy True:  그녀 곁에서 모두 다 물러나 이젠 조금씩 사나워진다.
greedy False:  그녀 앞에 손짓 발짓 저벅저벅 빛나는 검은 그림자 그림자.

Movie Sunny fine-tuning
greedy True:  그녀들, 일제히 수지를 쳐다본다.                                                                                            
greedy False:  그녀들을 보자 웃음바다가 된 공터. 언제 눈물바다가 됐냐는 듯 바라보는 나미. 진희 금옥 황 진희 금옥 축하한다.

Novel fine-tuning
greedy True:  그녀에게는 그 어떤 시련도 없었다.
greedy False:  그녀에게는 돈 몇 푼도 벌지 못하지만 그것으로 그녀는 제법 버석고 기름진 대지를 빨아들이고 가뿐하였다.


In [150]:
gpt_fine_tuning_test('너는', p=0.95)

GPT-2
greedy True:  너는 내 운명
greedy False:  너는 그 사람이 원하는 사람인데요

EXO lyrics fine-tuning
greedy True:  너는 날 미치게 만드는구나
greedy False:  너는 나 뿐 아니라 동경과 부러움의 대상이었지

Movie Sunny fine-tuning
greedy True:  너는 남 걱정 하지 말고 이거(동작 흉내) 턴 할때 팍팍 좀 돌란 말이야. (동작 흉내) 팍팍 좀 돌란 말이야.
greedy False:  너는? 황 진희랑 사귄다며? 나 보고 황 진희래.. 너 이 씨발년아. 니가 좀 맞아.

Novel fine-tuning
greedy True:  너는 나를 어떻게 생각하는지, 내가 왜 나를 미워해야 하는지, 내가 왜 남을 미워해야 하는지, 왜 남을 미워해야 하는지, 왜 남을 미워해야 하는지, 왜 남을 미워해야 하는지, 왜 남을 미워해야 하는지, 왜 남을 미워해야 하는지, 왜 남을 미워해야 하는지, 왜 남을 미워해야 하는지, 왜 남을 미워해야 하는지, 왜 남을 미워해야 하는지, 왜 남을 미워해야 하는지, 왜 남을 미워해야 하는지, 왜 남을
greedy False:  너는 어제의 내일이 두렵지 않은가!


In [151]:
gpt_fine_tuning_test('나는', p=0.95)

GPT-2
greedy True:  나는 이 모든 것을 알고 있다.
greedy False:  나는 노동조합이 파업으로 인한 고용불안 등을 사유화하지 못하도록 노조활동이 원칙대로 완수될 수 있도록 노력할 것이다.

EXO lyrics fine-tuning
greedy True:  나는 너를 사랑한 것이 아니라 널 사랑한 것이다
greedy False:  나는 여기서 멈추지 않고, 다시 나에게로 돌진하지 않을 수 없다.

Movie Sunny fine-tuning
greedy True:  나는 너를 안다는 듯 묘한 미소를 띄며 노래를 흥얼거리는 춘화. 살짝 떨리는 손. 살짝 떨리는 손. 살짝 떨리는 손. 살짝 떨리는 손.
greedy False:  나는 괜찮다니깐. (시위대 보며) 누가 뭐래? 폭력 시위나 하자고. (시위대 보며) 지금 이 시간부터 총동원해 완전 초토화 합니다.

Novel fine-tuning
greedy True:  나는 그 여학생에게 반항 한번 못해 본 것 같다.
greedy False:  나는 니가 자기를 좋아하든 말든 욕을 먹어도 내색조차 못하는데...... 무슨 병인가.”
